# Cross-Platform Estimation of Quantum States

*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

Direct Fidelity Estimation (DFE) proposes a method to estimate fidelity between ideal and actual pure state. However, it has two problems, on the one hand, it can not work when ideal state is unknown, on the other hand, it does not apply to the mixed state. Cross-Platform Estimation (CPE) can solve these two problems well.

This tutorial introduces Cross-Platform Estimation (CPE) , covering its theory and implementation on [Baidu Quantum Platform](https://quantum.baidu.com/).

## Theory



Here we will introduce how to use CPE to estimate fidelity between two pure quantum states produced with two different platforms. Now, consider two platforms $\mathcal{S}_1$ and $\mathcal{S}_2$, and this two platform prepare the same $n$-qubit quantum states $\rho_1$ and $\rho_2$ with circuits $\mathcal{U}_1$ and $\mathcal{U}_2$, respectively. We show how to measure directly the overlap ${\rm Tr}[\rho_1 \rho_2]$ as follows.

1. Randomly select unitary $U = \otimes_{i=1}^n U_i$, where $U_i$ are sampled independently from a unitary $2$-design gate set.
2. Perform $\mathcal{S}_1$ and $\mathcal{S}_2$ projective measurements in a computational basis $\left\{|x\rangle\right\}$.
3. Estimate $${\rm Pr}(x, y) = {\rm Tr}\left[\left(U\otimes U \rho_1\otimes\rho_2 U^\dagger\otimes U^\dagger\right) |x\rangle\langle x|\otimes|y\rangle\langle y|\right],$$ where $x,y$ are the measurement result of $\mathcal{U}_1, \mathcal{U}_2$.
4. Repeat step 1 to 3 $K$ times, record $k$-th probability distribution as ${\rm Pr}^{(k)}(x, y)$, and compute the average probability distribution $$\overline{{\rm Pr}(x, y)} = \frac{1}{K}\sum_k {\rm Pr}^{(k)}(x, y). $$
5. Estimate the density matrix overlap ${\rm Tr}[\rho_1 \rho_2]$ with $${\rm Tr}[\rho_1\rho_2]=2^n\sum_{x,y}(-2)^{-\mathcal{D}[x,y]}\overline{{\rm Pr}(x, y)}, $$ where $\mathcal{D}[x,y]$ denotes Hamming distance between bit strings $x, y$.

The flow chart is as follows.

![CPE](./figures/cpe-chart.png "Figure 1: We complete CPE according to this flow chart.")


## Practice

We demonstrate CPE on the single qubit case.

First, we import the necessary libraries.

In [2]:
import QCompute
from Extensions.QuantumErrorProcessing.qcompute_qep.estimation.cpe_state import QuantumSnapshot, CPEState

Then we set up the quantum platforms we are interested in. Here we choose first and second qubit of ideal simulator. And you can also construct `QuantumSnapshot` with `read_quantum_snapshot()` method.

In [3]:
ideal_baidu1 = QuantumSnapshot(qc_name='Baidu ideal1',
                               qc=QCompute.BackendName.LocalBaiduSim2,
                               qubits=[0])
ideal_baidu2 = QuantumSnapshot(qc_name='Baidu ideal2',
                               qc=QCompute.BackendName.LocalBaiduSim2,
                               qubits=[1])

dev_list = [ideal_baidu1, ideal_baidu2]

Then we set the quantum program, which used to prepare target quantum state. The rest is simple, we initialize a `CPEState` instance, call the estimate procedure and obtain the fidelity between two platforms. Here, we set $K=100$ and visualize the result.

In [4]:
# Construct the circuit used to prepare target state
qp = QCompute.QEnv()
qp.Q.createList(1)

est = CPEState()
result = est.estimate(dev_list, qp, samples=100, shots=50, show=True)


Finally, we can save the quantum platform information as a file.

In [ ]:
ideal_baidu1.save_data()
ideal_baidu2.save_data()

## References
[1] Elben, Andreas, et al. "Cross-platform verification of intermediate scale quantum devices." [Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.124.010504) 124.1 (2020): 010504.